In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import cycle, islice
from lib.nyt_connections import get_connections_games
from lib.task import Task
from openai.types.chat.chat_completion import Choice
from typing import Iterable


games = get_connections_games()
games

[ConnectionsGame(board={'WET WEATHER': ConnectionsGroup(level=0, members=['HAIL', 'RAIN', 'SLEET', 'SNOW']), 'NBA TEAMS': ConnectionsGroup(level=1, members=['BUCKS', 'HEAT', 'JAZZ', 'NETS']), 'KEYBOARD KEYS': ConnectionsGroup(level=2, members=['OPTION', 'RETURN', 'SHIFT', 'TAB']), 'PALINDROMES': ConnectionsGroup(level=3, members=['KAYAK', 'LEVEL', 'MOM', 'RACECAR'])}, starting_board=[['SNOW', 'LEVEL', 'SHIFT', 'KAYAK'], ['HEAT', 'TAB', 'BUCKS', 'RETURN'], ['JAZZ', 'HAIL', 'OPTION', 'RAIN'], ['SLEET', 'RACECAR', 'MOM', 'NETS']], created_at=datetime.datetime(2023, 6, 12, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), id=1, name='Connections #1'),
 ConnectionsGame(board={'FOOTWEAR': ConnectionsGroup(level=0, members=['BOOT', 'LOAFER', 'PUMP', 'SNEAKER']), 'UNITS OF LENGTH': ConnectionsGroup(level=1, members=['FOOT', 'LEAGUE', 'MILE', 'YARD']), 'MAGAZINES': ConnectionsGroup(level=2, members=['ESSENCE', 'PEOPLE', 'TIME', 'US']), 'LETTER HOMOPHONES': ConnectionsGroup(level=3, members=['ARE', 'Q

In [ ]:
prompts = [
    "Find groups of four items that share something in common. Output them in the following format: four total lines. On each line, there should be four comma-separated items. No additional text (like group titles or descriptions) should be in the output. Also, there should not be anything in your output before or after the solution.",
    "Group words that share a common thread. There are four words for each common thread. Output them in the following format: four total lines. On each line, there should be four comma-separated items. No additional text (like group titles or descriptions) should be in the output. Also, there should not be anything in your output before or after the solution.",
    "This is a puzzle. Create four groups of four. Words in each group fit under a specific category. Some categories might be defined by their use of wordplay (palindromes, homophones, adding or dropping letters and words) rather than the literal meanings of the words on the cards. Output them in the following format: four total lines. On each line, there should be four comma-separated items. No additional text (like group titles or descriptions) should be in the output. Also, there should not be anything in your output before or after the solution.",
]


def get_tasks() -> Iterable[Task]:
    for game, (prompt, lowercase) in zip(
        cycle(games),
        cycle((prompt, lowercase) for prompt in prompts for lowercase in [True, False]),
    ):

        def grader(choice: Choice) -> float:
            assistant_content = choice.message.content
            assert assistant_content is not None
            assistant_content = assistant_content.strip()
            groups = [
                frozenset(word.strip().upper() for word in line.split(","))
                for line in assistant_content.split("\n")[-len(game.board) :]
            ]
            reward = 0
            for _, group_contents in game.board.items():
                if set(group_contents.members) in groups:
                    reward += 1 / len(game.board)
            return reward

        yield Task(
            messages=[
                {
                    "role": "user",
                    "content": f"{prompt}\nWords:\n\n{"\n".join(word.lower() if lowercase else word for row in game.starting_board for word in row)}",
                }
            ],
            grader=grader,
        )


tasks = list(islice(get_tasks(), len(games) * 2))
distill_tasks = tasks[:436]
val_tasks = tasks[436:508]
test_tasks = tasks[508 : len(games)]
train_tasks = tasks[len(games) : len(games) + 436]
len(distill_tasks), len(val_tasks), len(test_tasks), len(train_tasks)

(436, 72, 92, 436)